In [7]:
#Step 1: Import Libraries and Load Data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [25]:
# Set global styles for better visualization
sns.set(style="whitegrid")

In [9]:
# Load datasets
customers = pd.read_csv(r"C:\Users\roxna\Downloads\Data_Intern\Customers.csv")
products = pd.read_csv(r"C:\Users\roxna\Downloads\Data_Intern\Products.csv")
transactions = pd.read_csv(r"C:\Users\roxna\Downloads\Data_Intern\Transactions.csv")

In [27]:
def display_info():
    print("Customers Dataset Info:")
    print(customers.info())
    print("\nProducts Dataset Info:")
    print(products.info())
    print("\nTransactions Dataset Info:")
    print(transactions.info())

display_info()

Customers Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None

Products Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None

Transactions Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   C

In [29]:
# Merge datasets for comprehensive analysis
merged_data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")
print("\nMerged Dataset Sample:")
print(merged_data.head())


Merged Dataset Sample:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  Comfort

In [31]:
# Check for missing values
print("\nMissing Values:")
print(merged_data.isnull().sum())


Missing Values:
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price_x            0
CustomerName       0
Region             0
SignupDate         0
ProductName        0
Category           0
Price_y            0
dtype: int64


In [33]:
# Drop missing values for simplicity
merged_data.dropna(inplace=True)
# Convert date columns to datetime
merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'])

In [41]:
# Churn analysis: Identify inactive customers
recent_date = merged_data['TransactionDate'].max()
merged_data['DaysSinceLastPurchase'] = (recent_date - merged_data['TransactionDate']).dt.days
churn_threshold = 180  # Days since last purchase to define churn
churned_customers = merged_data[merged_data['DaysSinceLastPurchase'] > churn_threshold]['CustomerID'].unique()
print(f"Number of churned customers: {len(churned_customers)}")

Number of churned customers: 185


In [43]:
# Step 3: Lookalike Model

# Feature engineering for customer behavior analysis
customer_features = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",
    "Quantity": "sum",
    "Price_x": "mean",
    "DaysSinceLastPurchase": "min"
}).reset_index()

In [47]:
# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.iloc[:, 1:])
# Compute similarity matrix
similarity_matrix = cosine_similarity(scaled_features)
# Generate lookalike recommendations
lookalikes = {}
for idx, customer_id in enumerate(customer_features["CustomerID"]):
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:4]
    lookalikes[customer_id] = [(customer_features.iloc[i[0]]["CustomerID"], i[1]) for i in similarity_scores]
# Format lookalike data for CSV
lookalike_data = []
for cust_id, recommendations in lookalikes.items():
    for rec_cust_id, score in recommendations:
        lookalike_data.append([cust_id, rec_cust_id, score])

lookalike_df = pd.DataFrame(lookalike_data, columns=["CustomerID", "SimilarCustomerID", "SimilarityScore"])
lookalike_df.to_csv(r"C:\Users\roxna\Downloads\Data_Intern\Lookalike.csv", index=False)